From Geo-Python 2019
Exercise 6, optional question 4 
https://github.com/geo-python-2019/Exercise-6/blob/c2e4b3966aff47531c6a03583752be0c577cd134/Exercise-6-problem-4.ipynb

Problem 4

- Retrieve daily summary data from 1959-2018 for Sodankyla Lokka from NOAA Climate Data Online Search
- Calculate average temperature using columns TMAX and TMIN
- Calculate the temperature anomalies in Sodankyla
- Calculate the monthly temperature differences between Sodankyla and Helsinki stations. Compare summer mean temperatures

In [ ]:
import pandas as pd

filepath = r"C:\Users\ash c\Desktop\python\1985608.txt"
data = pd.read_csv(filepath, delim_whitespace=True, usecols=["STATION", "DATE", "PRCP", "TMAX", "TMIN"], na_values=['*', '**', '***', '****', '*****', '******'], skiprows=[1])

In [ ]:
for idx in data:
    data["TAVG"] = (data["TMIN"] + data["TMAX"]) / 2

In [ ]:
def tempconvert(temp):
    output_temp = (temp - 32) / 1.8
    return output_temp

In [ ]:
data["CAVG"] = data["TAVG"].apply(tempconvert).astype(int64)
data["TIMESTR"] = data["DATE"].astype(str)
data["TIMESTAMP"] = pd.to_datetime(data["TIMESTR"])
data["MONTH"] = data["TIMESTAMP"].dt.month
data["YEAR"] = data["TIMESTAMP"].dt.year


In [ ]:
select = data.filter(["CAVG", "YEAR", "MONTH"])

monthlydata = select.groupby(["YEAR","MONTH"])
bymonth = select.groupby(["MONTH"])

In [ ]:
meancol = ["CAVG"]
monthlyavg = pd.DataFrame()
for key, group in monthlydata:
    monthmean = group[meancol].mean()
    monthmean["YRMO"] = key
    monthlyavg = monthlyavg.append(monthmean, ignore_index=True)

In [ ]:
timestamp = pd.DataFrame(monthlyavg["YRMO"].values.tolist(), columns=["YEAR", "MONTH"])
monthlyavg["YEAR"] = timestamp["YEAR"]
monthlyavg["MONTH"] = timestamp["MONTH"]

In [ ]:
reftemps = pd.DataFrame()
for key, group in bymonth:
    reftemp = group[meancol].mean()
    reftemps = reftemps.append(reftemp, ignore_index=True)
    
reftemps["MONTH"] = [1, 2, 3, 4, 5, 6, 7, 8, 9 , 10, 11, 12]

In [ ]:
diff = monthlyavg.merge(reftemps, on="MONTH")
diff["DIFF"] = diff["CAVG_x"] - diff["CAVG_y"]

newcols = {"CAVG_x": "OBS TEMP", "CAVG_y": "REFTEMP", "DIFF": "ANOMALY"}
diff = diff.rename(columns=newcols)
diff = diff[["YEAR", "MONTH", "OBS TEMP", "REFTEMP", "ANOMALY"]]
diff = diff.sort_values(["YEAR", "MONTH"], ascending=True)
diff = diff.round(1)

In [ ]:
filepath_hsk = r"C:\Users\ash c\Desktop\python\helsinki.csv"
hsk = pd.read_csv(filepath_hsk, usecols=["YEAR", "MONTH", "TEMP_C", "REFTEMP_C", "DIFF"])
hsknewcols = {"TEMP_C": "OBS TEMP", "REFTEMP_C": "REFTEMP", "DIFF": "ANOMALY"}
hsk = hsk.rename(columns=hsknewcols)

In [ ]:
climate = diff.merge(hsk, on=["YEAR","MONTH"])
climate_newcols = {"OBS TEMP_x": "SDK TEMP", "REFTEMP_x": "SDK REFTEMP", "ANOMALY_x": "SDK ANOMALY", "OBS TEMP_y": "HSK TEMP", "REFTEMP_y": "HSK REFTEMP", "ANOMALY_y": "HSK ANOMALY"}
climate = climate.rename(columns=climate_newcols)

In [ ]:
for idx in climate:
    climate["DIFFERENCE"] = (climate["SDK TEMP"] - climate["HSK TEMP"])
    
climate["DIFFERENCE"] = climate["DIFFERENCE"].abs()

In [ ]:
climate.to_csv("hsk_sdk.csv", sep=',', index=False, float_format="%.1f") 